In [1]:
%matplotlib widget

In [141]:
# %matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
# from itertools import product
import itertools

In [160]:
class Block:
    def __init__(self, dims=[32,32], density=None):
        self.dims = np.array(dims)
        self.rank = len(self.dims)
        self.vel = np.random.uniform(-1, 1, [self.rank])
        self.density = density if density else np.random.uniform(0, 1)
        self.subdivisions = 2
        self.children = np.empty([self.subdivisions]*self.rank, dtype=object)
        
    def divide(self, traverse=False, inherit=True):
        if np.min(self.dims) >= 2:
            for w in np.ndindex(self.children.shape):
                d = self.density if inherit else None
                self.children[w] = Block(dims=self.dims/self.subdivisions, density=d)
                if traverse:
                    self.children[w].divide(traverse=traverse, inherit=inherit)
        return self.children
    
    def extract(self, b):
        if b.children.any():
#             print(b.children)
            data = np.array([self.extract(b.children[i]) for i in np.ndindex(b.children.shape)])
#             print(data.shape)
#             rank = round(data.shape[0]**1/2)-1
            rank = round(np.product(data.shape)**(1/2))
#             shape = [2**rank]*2 + [-1]
            shape = np.array([rank]*2).astype(int)
            raw = [2,2]+([rank-1]*2)
#             raw = [r for r in raw if r!=1]
            v = np.array(raw).astype(int)
#             v = np.squeeze(v)
#             data = np.reshape(data, v)
            return np.reshape(data, tuple(shape))
        else:
            return b.density
    
    def simulate(self):
        new_children = np.empty_like(self.children)
#         if self.children.any():
        if self.children[(0,0)].children.any():
            for i in np.ndindex(self.children.shape):
                self.children[i].simulate()
        else:
            for i in np.ndindex(self.children.shape):
                new_children[i] = Block(density=self.children[i].density)
                for j in itertools.product([-1,0,1], repeat=2):
                    j = np.array(j)
                    if ([0]*2 < j).all() and (j < self.children.shape).all():
                        new_children[i].density += np.mean(self.children[i].density * j)
            self.children = new_children
    
    def render(self):
        canvas = self.extract(self).reshape(self.dims)
        fig, ax = plt.subplots()
        ax.imshow(canvas)
#         ax.show()
        return canvas
        
quadtree = Block()
quadtree.divide(traverse=True, inherit=False)
quadtree.render().shape
for i in range(10):
    quadtree.simulate()

# print([x.children for x in quadtree.children])
# print(quadtree.children[(0,0)].children)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …